In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
print(os.environ["LANGCHAIN_TRACING_V2"])

[API KEY]
sk-proj-2ALiWBzcJl4s9ri6EUJ6T3BlbkFJxbUDIanlzbIf6JMPE7o2
true


In [3]:
import torch
import os
import uuid

from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts.chat import PromptTemplate, ChatPromptTemplate

import numpy as np
import matplotlib.pyplot as plt

from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
import transformers

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from transformers import pipeline, AutoModel, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager

In [4]:
# 단계 1 : 데이터셋 로드

import json
# JSON 파일에서 데이터를 불러와 Document 객체로 복원
with open("/media/choi/HDD1/mmaction2/data/Korea_construction_standard/LHCS_qna_id.json", "r", encoding="utf-8") as f:
    documents_data = json.load(f)

documents_512 = [
    Document(page_content=doc["page_content"], metadata=doc["metadata"]) for doc in documents_data
]

print(f"문서의 페이지수 : {len(documents_512)}")

문서의 페이지수 : 1255


In [5]:
# 단계 2 : 임베딩 모델 로드
# EM_klue_nli = HuggingFaceEmbeddings(model_name = '/home/choi/Git/ConSRoBERTa/output/2024_2/klue_nli_top_2e_55.24/kor_multi_klue-2024-09-19_15-26-29e1e2')
# EM_open = OpenAIEmbeddings(model = 'text-embedding-3-large') # 최신 GPT4 유로 모델
# EM_klue_vanilla = HuggingFaceEmbeddings(model_name = 'klue/roberta-base')
EM_MNRL_MRL = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')
# embedding_models = [
#     EM_klue_nli,
#     EM_MNRL_MRL,
#     EM_open,
#     EM_klue_vanilla,
# ]

In [6]:
# top 5검색 결과와 답변 생성까지 저장
llm = ChatOpenAI(model_name='gpt-4o', temperature=0.0)

answer_prompt = ChatPromptTemplate.from_template(
    """
- <Question> 이후에 오는 질문은 <Text> 이후에 오는 문서와 관련된 질문이야.
- <Text> 이후의 문서를 참고해서 <Question> 이후에 오는 질문에 대한 답변을 생성해줘.
- 정확하고 간결하게 답변해줘
- 답변은 한글로 출력해줘


<Question> : {question}
<Text>: {input}
----
<Answer>:
    """
)

class AnswerParser(StrOutputParser):
    def parse(self, response):
        # "Question:" 이후의 텍스트를 추출
        if "Answer>:" in response:
            return response.split("Answer>:")[1].strip()
        return response.strip()
    
A_chain = answer_prompt | llm | AnswerParser()


In [32]:
# 5 단계 : 768d 를 자르고 임베딩한 후 여러 방법으로 합쳐서 하나의 768d 벡터 만들기
def Accuracy_test(documents, embed_model, model_name, chunk_size):
    
    # 1. Langchain의 RecursiveCharacterTextSplitter 사용
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)

    # 모델 불러오기 (HuggingFaceEmbeddings 사용)
    # embed_model = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')

    # 문서 예시 (documents_512에서 첫 번째 문서)
    for doc in documents:
        
        text = doc.page_content
        metadata = doc.metadata
        # 2. 임베딩 벡터와 관련된 기존 데이터 초기화
        for key in list(metadata.keys()):
            if 'embedding_vector' in key or 'num_chunks' in key:
                del metadata[key]  # 이전 임베딩 관련 데이터 삭제
                
        # 2. RecursiveCharacterTextSplitter로 문서를 128토큰 단위로 분할
        split_documents = splitter.split_text(text)

        # 3. 각 분할된 텍스트를 임베딩
        chunk_embeddings = [embed_model.embed_query(chunk) for chunk in split_documents]

##################################combined_embedding 한개만 쓰면됨################################################################

        # 4. 768 차원으로 결합하는 방식 (method에 따라 다름)
        if model_name == "768_Mean":
            combined_embedding = np.mean(chunk_embeddings, axis=0)
        elif model_name == "768_First":
            combined_embedding = np.concatenate([chunk[:768 // len(chunk_embeddings)] for chunk in chunk_embeddings], axis=0)
        elif model_name == "768_Each":
            num_chunks = len(chunk_embeddings)
            split_size = 768 // num_chunks  # 각 청크에서 가져올 크기 계산
            combined_embedding = []

            # 각 청크에서 순차적으로 부분을 가져오는 방식
            for i, chunk in enumerate(chunk_embeddings):
                start_index = i * split_size
                end_index = (i + 1) * split_size

                # 각 청크에서 해당 부분을 결합
                combined_embedding.append(chunk[start_index:end_index])

            combined_embedding = np.concatenate(combined_embedding, axis=0)

            # 결합된 벡터가 768차원이 안 되면 마지막 청크에서 부족한 부분을 가져와 채우기
            if combined_embedding.shape[0] < 768:
                remaining_size = 768 - combined_embedding.shape[0]
                combined_embedding = np.concatenate([combined_embedding, chunk_embeddings[-1][:remaining_size]], axis=0)

        else:
            raise ValueError("Invalid method specified.")

        # Metadata에 벡터 저장
        metadata[f'{model_name}embedding_vector'] = combined_embedding.tolist()
        metadata['num_chunks'] = len(chunk_embeddings)

        # print(f"Document ID: {metadata['id']} - Embedding created with {model_name}")


##################################combined_embedding 한개만 쓰면됨################################################################

    # 문서를 딕셔너리로 변환하여 JSON으로 저장
    documents = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]

    with open(f'/home/choi/Git/RAG_con_doc/langchain/{model_name}_{chunk_size}_results.json', 'w', encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=4)

In [ ]:
# 5 단계 : 768d 를 자르고 임베딩한 후 concatenation 하기
def Accuracy_test(documents, embed_model, model_name, chunk_size):
    
    # 1. Langchain의 RecursiveCharacterTextSplitter 사용
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)

    # 모델 불러오기 (HuggingFaceEmbeddings 사용)
    # embed_model = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')

    # 문서 예시 (documents_512에서 첫 번째 문서)
    for doc in documents:
        
        text = doc.page_content
        metadata = doc.metadata
        # 2. 임베딩 벡터와 관련된 기존 데이터 초기화
        for key in list(metadata.keys()):
            if 'embedding_vector' in key or 'num_chunks' in key:
                del metadata[key]  # 이전 임베딩 관련 데이터 삭제
                
        # 2. RecursiveCharacterTextSplitter로 문서를 128토큰 단위로 분할
        split_documents = splitter.split_text(text)

        # 3. 각 분할된 텍스트를 임베딩
        chunk_embeddings = [embed_model.embed_query(chunk) for chunk in split_documents]

##################################combined_embedding 한개만 쓰면됨################################################################

        # 4. 768 차원으로 결합하는 방식 (method에 따라 다름)
        if model_name == "768_Mean":
            combined_embedding = np.mean(chunk_embeddings, axis=0)
        elif model_name == "768_First":
            combined_embedding = np.concatenate([chunk[:768 // len(chunk_embeddings)] for chunk in chunk_embeddings], axis=0)
        elif model_name == "768_Each":
            num_chunks = len(chunk_embeddings)
            split_size = 768 // num_chunks  # 각 청크에서 가져올 크기 계산
            combined_embedding = []

            # 각 청크에서 순차적으로 부분을 가져오는 방식
            for i, chunk in enumerate(chunk_embeddings):
                start_index = i * split_size
                end_index = (i + 1) * split_size

                # 각 청크에서 해당 부분을 결합
                combined_embedding.append(chunk[start_index:end_index])

            combined_embedding = np.concatenate(combined_embedding, axis=0)

            # 결합된 벡터가 768차원이 안 되면 마지막 청크에서 부족한 부분을 가져와 채우기
            if combined_embedding.shape[0] < 768:
                remaining_size = 768 - combined_embedding.shape[0]
                combined_embedding = np.concatenate([combined_embedding, chunk_embeddings[-1][:remaining_size]], axis=0)

        else:
            raise ValueError("Invalid method specified.")

        # Metadata에 벡터 저장
        metadata[f'{model_name}embedding_vector'] = combined_embedding.tolist()
        metadata['num_chunks'] = len(chunk_embeddings)

        # print(f"Document ID: {metadata['id']} - Embedding created with {model_name}")


##################################combined_embedding 한개만 쓰면됨################################################################

    # 문서를 딕셔너리로 변환하여 JSON으로 저장
    documents = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]

    with open(f'/home/choi/Git/RAG_con_doc/langchain/{model_name}_{chunk_size}_results.json', 'w', encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=4)

In [44]:
# 검색 함수 구현 (L2 거리 기반)
def search_documents(query_embedding, documents):
    # 검색할 모든 문서의 벡터와 L2 거리 계산
    distances = []
    for doc in documents:
        doc_num_chunks = doc.metadata['num_chunks']
        doc_vector = np.array(doc.metadata[f'{model_name}embedding_vector'])  # metadata에 저장된 벡터 사용
        distance = np.linalg.norm(query_embedding - doc_vector)  # L2 거리 계산
        std_dev = np.std(query_embedding - doc_vector)
        distances.append((doc, distance * doc_num_chunks**0.5))
    
    # 거리 기준으로 가장 가까운 문서 5개 찾기
    distances.sort(key=lambda x: x[1])  # 거리가 작은 순서대로 정렬
    top_5 = distances[:5]  # 상위 5개의 문서
    
    # 상위 5개 문서 ID를 추출해서 metadata에 추가
    top_5_ids = [doc.metadata['id'] for doc, _ in top_5]
    
    return top_5, top_5_ids

# 정확도 계산 함수
def calculate_accuracy(documents, embed_model):
    correct_count = 0

    for doc in documents:
        question = doc.metadata['question']  # 질문을 metadata에서 불러옴
        correct_doc_id = doc.metadata['id']  # 정답 문서 ID

        # 질문 임베딩
        query_embedding = embed_model.embed_query(question)

        # 검색
        top_5, top_5_ids = search_documents(query_embedding, documents)
        
        # 상위 5개 문서 중 가장 가까운 문서의 ID와 정답 ID 비교
        if top_5_ids[0] == correct_doc_id:
            correct_count += 1

        # 검색된 문서에 상위 5개의 ID를 metadata에 저장
        doc.metadata['top_5_ids'] = top_5_ids

    # 정확도 계산 (정답 문서와 일치한 비율)
    accuracy = correct_count / len(documents)
    print(f"정확도: {accuracy * 100:.2f}%")

    # 업데이트된 문서를 저장
    documents_dict = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]
    with open('/home/choi/Git/RAG_con_doc/langchain/updated_768_Each_results.json', 'w', encoding='utf-8') as f:
        json.dump(documents_dict, f, ensure_ascii=False, indent=4)
# JSON 파일에서 데이터를 불러와 Document 객체로 변환
model_name = '768_Each'

with open(f"/home/choi/Git/RAG_con_doc/langchain/{model_name}_results.json", "r", encoding="utf-8") as f:
    documents = json.load(f)

documents = [
    Document(page_content=doc["page_content"], metadata=doc["metadata"]) for doc in documents
]

print(f"문서의 페이지 수: {len(documents)}")
# 예시 실행
calculate_accuracy(documents, embed_model)

문서의 페이지 수: 1255
정확도: 3.82%


In [33]:
embed_model = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')
embedding_models2 = [
    # ('EM_klue_nli', EM_klue_nli),
    # (embed_model, '768_Mean'),
    # (embed_model, '768_First'),
    (embed_model, '768_Each'),
]
for embed_model, model_name in embedding_models2:
    Accuracy_test(documents_512, embed_model, model_name, 256)
# Accuracy_test(documents_512, embed_model, "768_Mean")

In [23]:
# NDCG와 MRR 점수 비교하고 결과 top5 랑 답변 결과까지 저장하기
import json
from sklearn.metrics import ndcg_score

# FAISS로 검색 후 상위 5개의 검색 결과를 저장하고, NDCG@5, MRR@5를 계산하는 함수
def evaluate_and_save_results(embedding_model, documents, output_file, A_chain):
    
    vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)
    
    results = []
    ndcg_scores = []
    mrr_scores = []

    for doc in documents:
        question = doc.metadata['question']
        correct_id = doc.metadata['id']
        correct_answer = doc.metadata['answer']

        # FAISS로 상위 5개 검색
        retrieved_results = vectorstore.similarity_search(query=question,k=5)

        # 검색된 문서들의 ID를 저장
        retrieved_ids = [result.metadata['id'] for result in retrieved_results]

        # 상위 1개의 검색 결과로 답변 생성
        top_document = retrieved_results[0].page_content
        generated_answer = A_chain.invoke({
            'question': question,
            'input': top_document
        })

        # 정답 ID를 NDCG와 MRR 계산용으로 변환
        y_true = [1 if retrieved_id == correct_id else 0 for retrieved_id in retrieved_ids]
        y_score = [5, 4, 3, 2, 1]  # 순위에 따른 가중치

        # NDCG@5 계산
        ndcg = ndcg_score([y_true], [y_score], k=5)
        ndcg_scores.append(ndcg)

        # MRR@5 계산
        try:
            rank = retrieved_ids.index(correct_id) + 1
            mrr = 1 / rank
        except ValueError:
            mrr = 0  # 정답이 top 5 안에 없으면 MRR은 0
        mrr_scores.append(mrr)

        # 결과를 저장할 데이터 구성
        results.append({
            'question': question,
            'correct_id': correct_id,
            'retrieved_ids': retrieved_ids,
            'correct_answer': correct_answer,
            'generated_answer': generated_answer,  # 쉼표 누락 수정
            'ndcg@5': ndcg,
            'mrr@5': mrr
        })

    # 결과를 JSON 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    # NDCG@5, MRR@5 평균 출력
    print(f"Average NDCG@5: {sum(ndcg_scores) / len(ndcg_scores)}")
    print(f"Average MRR@5: {sum(mrr_scores) / len(mrr_scores)}")

In [ ]:
# 간단한 테스트
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
# 임베딩 모델 로드 (파인튜닝된 모델)

# Accuracy_test(documents_512, EM_klue_nli)
# Accuracy_test(documents_512, EM_open)
# Accuracy_test(documents_512, EM_klue_vanilla)
# output_path = '/home/choi/Git/RAG_con_doc/langchain/MNRL_MRL.json'
# evaluate_and_save_results(embed_model, documents_512_sample, output_path, A_chain)
embedding_models2 = [
    # ('EM_klue_nli', EM_klue_nli),
    ('EM_MNRL_MRL', EM_MNRL_MRL),
    ('EM_open', EM_open),
    ('EM_klue_vanilla', EM_klue_vanilla),
]

for model_name, embed_model in embedding_models2:
    output_path = f'/home/choi/Git/RAG_con_doc/langchain/{model_name}_results.json'
    print(f"Evaluating model: {model_name}")
    evaluate_and_save_results(embed_model, documents_512, output_path, A_chain)

Evaluating model: EM_MNRL_MRL
Average NDCG@5: 0.7454216677828364
Average MRR@5: 0.7047144754316079
Evaluating model: EM_open
